## CNN attack on 5px Analog Linearalized Data using Single CNN

The goal of this notebook is to correctly preprocess the given data as tensors that can be used to train a single simple CNN and evaluate its results.

In [ ]:
# dataloader.py
# Necessary imports
from torch.utils.data import Dataset, DataLoader
import numpy as np
import os
import re

## Common rules
1) The BIT numbers are arranged in descending order
    1) Thus, in a 8-bit ADC, bit 7 is the MSB and bit 0 is the LSB.
2) The ADC numbers are arranged in descending order.
    1) Thus, in a 5-pixel case, the ADC with adc_num 4 stores bits [39-32], and ADC with adc_num 0 stores bits [7-0].

#### Hyperparameter - File Settings
Hyperparameters used to define the nature of the power trace files and values used for creating the dataloaders.

Double-check if these values match the format of the power trace files; failure to do so may cause unforeseen errors which are extremely difficult to debug.

In [ ]:
''' 
Hyperparameter used to define settings of power trace files.
    1) adc_num: number of pixels dataloader we are training/testing is based on. This number of ADCs are created.
    2) split_digital: BOOLEAN value. Set to 'True' if the dataloaders need the digital output values of individual ADCs.
    Used to save computational power and prevent repetitive operations.
    3) normalized_digital: BOOLEAN value. Set to 'True' if the file names provide normalized digital values.
    4) train_batch: training dataloader batch size.
    5) e_exp: hyperparameter used in normalizing values.
    All values are normalized by pow(10, -1*e_exp) to save computational resources.
'''
adc_num = 5
adc_bitwidth = 8
split_digital = True
normalized_digital = True
train_batch = 256
e_exp = 3

#### Hyperparameter - Trace File Directories
All power trace files are assumed to be saved in a folder called 'trace_files' within the root directory of the notebook.

Define names of all folders within 'trace_files' to be added to the dataloaders within this hyperparameter.

In [ ]:
''' 
Hyperparameter used to desginate trace folders to be loaded.
    1) sub_folder: BOOLEAN value that is used when trace folders are stored in a specific
    folder within the parent directory.
    2) sub_folder_name: name of sub-folder
    ex) If trace folders are stored in /trace_folders/analog_5px, set sub_folder to True
    and sub_folder_name to 'analog_5px'
    3) train_trace_folder_names: Add names of folders used for TRAINING dataset to this array.
    4) test_trace_folder_names: Add names of folders used for TESTING dataset to this array.
    5) trace_type: string that defines the type of trace file to be used. Default is "lin" for linearized.
    6) file_pattern: RegEx of file names to be checked. Number of groups MUST match 'adc_num'
'''
sub_folder = True
sub_folder_name = 'analog_5px'
train_trace_folder_names = ['analog_5px_tt_px']
test_trace_folder_names = ['analog_5px_tt_pm']
trace_type = "lin"
file_pattern = trace_type + "_s\d+_([0-9]+\.[0-9]+)_([0-9]+\.[0-9]+)_([0-9]+\.[0-9]+)_([0-9]+\.[0-9]+)_([0-9]+\.[0-9]+)\.txt"


### Creating dataloaders
Used files from Kareem's GitHub repo. I may have made mistakes in sampling the data, so feel free to change anything that has been configured incorrectly.

#### Helper Functions
process_string: Function that processes trace string values to correct format float64.

In [ ]:
''' 
Function that processes trace string values to correct format float64.
It normalizes the values using 'e_exp' hyperparameter and returns correct float64.
Inputs:
    1) strip_val: value before 1e
    2) e_val: 1e exponent value
    ex) if input string is "-2.17498915e-03", strip_val = "-2.17498915", e_val = "-03"
    3) e_exp: defined hyperparameter. Mormalizes all values by pow(10, -1*e_exp)
Returns:
    1) new_number: normalized float64 of string
'''
def process_string(strip_val, e_val, e_exp=e_exp):
    # e_diff: exponent difference from base -8 value
    e_diff = int(e_val) + e_exp
    is_neg = strip_val.startswith('-')

    # Remove negative sign for processing
    if is_neg:
        strip_val = strip_val[1:]
    
    int_part, dec_part = strip_val.split('.')
    new_number = int_part + dec_part

    if e_diff < 0:
        new_number = '0' * abs(e_diff - 1) + new_number
        new_number = '0.' + new_number
    else:
        if e_diff + 1 < len(new_number):
            new_number = new_number[:e_diff + 1] + '.' + new_number[e_diff + 1:]
        else:
            new_number = new_number.ljust(e_diff + 1, '0')
    new_number = np.float64(new_number)

    return new_number * -1 if is_neg else new_number

#### Trace Classes
1) TraceDataset(Dataset): creates dataset of TRACE files

In [ ]:
# dataloader.py
# Creates dataset with given traces
class TraceDataset(Dataset):
    # cached_traces, trace_list = used to store already created traces
    # reused based on digital value, prevents rep calls on load_traces
    cached_traces = {}
    trace_list    = []

    # file_list: list of FILE NAMES that have been converted
    # cache: actual traces saved that can be reused
    def __init__(self, file_list, cache=True):
        self.file_list = file_list
        self.cache     = cache
    
    def __len__(self):
        return len(self.file_list)

    # input: index, used for finding file in file_list
    def __getitem__(self, index):
        fname, fpath, label = self.file_list[index]
        label = self.process_label(label)

        if self.cache and fname in self.cached_traces:
            return self.cached_traces[fname], label
        else:
            return self.load_trace(fname, fpath), label

    def get_info(self, index):
        return self.file_list[index]

    # opens single trace file, creates valu_arr, patches as tensor
    def load_trace(self, fname, fpath):
        with open(fpath, 'r') as file:
            header = file.readline()
            #time_arr = []
            valu_arr = []
            # Fixed error of float32 incorrectly translating values
            for line in file.readlines():
                time, value = line.strip().split()
                # Edge case: value is "-0.00000000e+00" or "0.00000000e+00"
                # Add more edge cases if needed
                if value in ["-0.00000000e+00", "0.00000000e+00"]:
                    valu_arr.append(np.float64(0))
                else:
                    try:
                        match = re.search(r"(?<=e-)\d+", value)
                        if match:
                            if value[0] == "-":
                                strip_val = value[0:11]
                                strip_val_e = value[12:15]
                            else:
                                strip_val = value[0:10]
                                strip_val_e = value[11:14]
                            '''
                            # Debugging scripts; do not erase
                            print(f"\tstrip_val: {strip_val}")
                            print(f"\tstrip_val_e: {strip_val_e}\n")
                            new_val = process_string(strip_val, strip_val_e)
                            print(f"\tProcessed: {new_val}")
                            print(f"\tFloat32 of processed: {np.float32(new_val)}\n")
                            '''
                            valu_arr.append(process_string(strip_val, strip_val_e))
                    except ValueError as e:
                        print(f"Error parsing value '{value}': {e}")
        
        trace = np.array(valu_arr, dtype=np.float32)
        '''
        # Debugging scripts; do not erase
        print(f"\t{trace}")
        print(f"\tfname: {fname}")
        print(f"\tfpath: {fpath}\n")
        '''

        if self.cache: 
            self.cached_traces[fname] = trace
            self.trace_list.append(trace)

        return trace
    
    # label = PURE digital value as array
    def process_label(self, label):
        return label

    def cache_all(self):
        assert self.cache == True
        
        print("Caching all traces")
        for fname, fpath, label in self.file_list:
            self.load_trace(fname, fpath)
        print("DONE Caching all traces")

#### Trace Classes
2) TraceDatasetBW(TraceDataset): takes a TraceDataset, labels them by bit
3) TraceDatasetBuilder: creates final Dataloader with traces and labels

In [ ]:
class TraceDatasetBW(TraceDataset):
    # bit_select = 0-7, 0 = LSB, 7 = MSB
    # adc_select = 0-4, 0 = ADC storing LSB, 4 = ADC storing MSB
    def __init__(self, file_list, bit_select, adc_select, cache=True):
        # if split_digital, need to create SEPARATE dataloaders PER ADC
        if split_digital:
            self.adc_num = adc_select
            self.bit_mask =  1 << bit_select
        else:
            self.adc_num = 0
            self.bit_mask =  1 << bit_select + adc_select * 8

        super().__init__(file_list, cache=cache) 
    
    # Uses bitwise on COMBINED label
    def process_label(self, label):
        # if split_digital, label = array
        if split_digital:
            try:
                label_num = label[self.adc_num]
            except IndexError:
                print(f"\tInvalid index; this may be caused due to bad hyperparameters.")
                print(f"\tadc_num: {adc_num}")
                print(f"\tsplit_digital: {split_digital}")
                print(f"\tnormalized_digital: {normalized_digital}")
        else:
            label_num = label
        return 1 if label_num & self.bit_mask else 0

class TraceDatasetBuilder:
    def __init__(self, adc_bitwidth=8, cache=True):
        self.file_list = []
        self.cache = cache
        self.adc_bits = adc_bitwidth

        self.dataset = None
        self.dataloader = None
        self.datasets = []
        self.dataloaders = []

    def add_files(self, directory, format, label_group):
        ''' Builds list of powertrace files
        Inputs:
            directory   : folder to search for files
            format      : regular expression to match filenames
            label_group : group index for digital output label corresponding to trace
        Outputs:
            list        : [(file_name, file_path, label) ... ]
        '''
        format = re.compile(format)
        fnames = os.listdir(directory)

        for fname in fnames:
            if match := format.match(fname):
                fpath = os.path.join(directory, fname)
                # IF split_digital, return ARRAY of digital values
                # returns: [[adc_num digital values], ...] (2D array)
                # ORDER: MSB values FIRST
                if split_digital:
                    # dvalue: ordered by FILE NAMING order
                    # if normalized, multiply 256 to get original value
                    if normalized_digital:
                        dvalue = [int(np.float64(i) * 256) for i in match.groups()]
                    # else, append as int
                    else:
                        dvalue = [int(i) for i in match.groups()]
                # ELSE, return ARRAY of SINGLE digital values
                # returns: [combined digital value, ...] (1D array)
                else:
                    dvalue = [0]
                    # if normalized, multiply 256 to get original value
                    if normalized_digital:
                        for i in match.groups():
                            dvalue[0] = dvalue[0] * 256 + int(np.float64(i) * 256)
                    # else, append as int 
                    else:
                        for i in match.groups():
                            dvalue[0]  = dvalue[0] * 256 + int(i)
                
                self.file_list.append((fname, fpath, dvalue))

    def build(self):
        # dataset = TraceDataset, trace - digital value label ONLY
        self.dataset = TraceDataset(self.file_list, cache=self.cache)
        # Append dataloaders IN LSB ORDER; dataloader[0] = adc[0], bit[0]
        # dataloader[39] = adc[4], bit[7]
        # adc_dataloader[adc_num] = [dataloader[adc_num*8+0], dataloader[adc_num*8+1], ..., dataloader[adc_num*8+7]]
        for adc in range(adc_num):
            for bit in range(self.adc_bits):
                self.datasets.append(TraceDatasetBW(self.file_list, bit, adc, cache=self.cache))
                
        if self.cache:
            self.dataset.cache_all()

    def build_dataloaders(self, **kwargs): # batch_size=256, shuffle=True
        self.dataloader = DataLoader(self.dataset, **kwargs)
        self.dataloaders = [DataLoader(dataset, **kwargs) for dataset in self.datasets]

#### Create separate dataloaders - Training and Testing
Create two separate dataloaders from the provided files.

Need some clarification on the specific logic for choosing files(256 per training/testing, mixing both, etc.)

There are also some prerequisits for the dataloaders:
1) all files should be the same length. If there are 2610 values in one file, all files should have 2610.
2) each dataloader should have at least one trace mapped per digital value. I am not completely sure on the impact of selecting less than 256 traces per dataloader, but based on previous attempts on using partial data, I am guessing it is necessary.
3) a good ratio between training and testing is 7:3. We will need more data to make this possible.

In [ ]:
# Create dataloaders
pwd = os.getcwd()
trace_folder_dir = os.path.join(pwd, 'trace_files')
if sub_folder == True:
    trace_folder_dir = os.path.join(trace_folder_dir, sub_folder_name)

train_builder = TraceDatasetBuilder(adc_bitwidth=8, cache=True)
test_builder = TraceDatasetBuilder(adc_bitwidth=8, cache=True)

# Create TRAINING dataloader
try:
    for folder_name in train_trace_folder_names:
        train_builder.add_files(os.path.join(trace_folder_dir, folder_name), file_pattern, 0)
except FileNotFoundError:
    print(f"The folder does not exist: {folder_name}")
    print(f"Check the following settings and try again:")
    print(f"\tsub_folder: {sub_folder}")
    print(f"\tsub_folder_name: {sub_folder_name}")
    print(f"\ttrain_trace_folder_names: {train_trace_folder_names}")
    print(f"\ttest_trace_folder_names: {test_trace_folder_names}")
    print(f"\tfile_pattern: {file_pattern}")
    raise
train_builder.build()
print(len(train_builder.datasets[0]))
train_builder.build_dataloaders(batch_size=train_batch, shuffle=True)

# Create TESTING dataloader
try:
    for folder_name in test_trace_folder_names:
        test_builder.add_files(os.path.join(trace_folder_dir, folder_name), file_pattern, 0)
except FileNotFoundError:
    print(f"The folder does not exist: {folder_name}")
    print(f"Check the following settings and try again:")
    print(f"\tsub_folder: {sub_folder}")
    print(f"\tsub_folder_name: {sub_folder_name}")
    print(f"\ttrain_trace_folder_names: {train_trace_folder_names}")
    print(f"\ttest_trace_folder_names: {test_trace_folder_names}")
    print(f"\tfile_pattern: {file_pattern}")
    raise
test_builder.build()
test_builder.build_dataloaders(batch_size=train_batch, shuffle=True)

## Training CNNs - Native CNN
Using a native CNN, we train each CNN until all of them reaches an accuracy of 1.0.

I'm not sure if aiming for an accuracy of 1.0 is beneficial, as it is just overfitting the model to the training data. A more realistic value may be 0.99, but will set it to 1.0 for the current simulated environment.

The training function automatically reduces the learning rate used in Adam based on target accuracy. Currently testing different values and decrease rates. 

### Hyperparameters

1) def_lr: Default learning rate. Change this value to test different learning rate values.
2) max_epoch: Limit of maximum training epochs; training ends if it reaches this amount of epochs.

In [ ]:
# Main hyperparameters
def_lr = 1e-4
max_epoch = 100000

### Looking for optimal CNN parameters

Paramerize components of CNN and find best overall hyperparameters.

In [ ]:
# Imports
import json
import os
import sys
import time
import hashlib
import datetime
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import re

from copy import copy

import argparse

#### CNN Hyperparameter format

1. Convolutional Layer (Conv1d)
Format:
```
conv_[# of conv layer]_[# of out_channels]_[kernel_size #]_[stride #]_[padding #]_[dilation #]
```
Example:
```python
self.conv1 = nn.Conv1d(in_channels=1, out_channels=8, kernel_size=5, stride=1, padding=2, dilation=1)
# Naming: conv_1_1_8_5_1_2_1
```
NOTE: Padding mode = `'zeros'`

2. Max Pooling Layer (MaxPool1d)
Format:
```
maxpool_[# of pool layer]_[kernel_size #]_[stride #]_[padding #]_[dilation #]
```
Example:
```python
self.pool1 = nn.MaxPool1d(kernel_size=3, stride=3, padding=0, dilation=1)
# Naming: maxpool_1_3_3_0_1
```

---

3. Fully Connected Layer (Linear)
Format:
```
linear_[# of fc layer]_[in_features #]_[out_features #]
```
Example:
```python
self.fc1 = nn.Linear(500, 500)
# Naming: linear_1_500_500
```

In [ ]:
# Hyperparameter update testing area
# Change values within this section to attempt training on CNN

# default for both conv and maxpool: padding=0, dilation=1
# conv_[# of conv layer]_[# of out_channels]_[kernel_size #]_[stride #]_[padding #]_[dilation #]
# maxpool_[# of pool layer]_[kernel_size #]_[stride #]_[padding #]_[dilation #]
# linear_[# of fc layer]_[in_features #]_[out_features #]
conv_param = "conv_2_5_3_1_0_1"
maxpool_param = "maxpool_5_5_0_1"
linear_param = "linear_3_100_100"

### Creating CNNs based on defined parameters

In [ ]:
class DynamicCNN(nn.Module):
    def __init__(self, conv_params, maxpool_params, linear_params, input_length=3001):
        super(DynamicCNN, self).__init__()
        self.layers = nn.ModuleList()
        in_channels = 1  # Initial input channel
        final_size = input_length
        # print(final_size)

        # Parse convolutional layer parameters
        try:
            conv_layer_num, out_channels, conv_kernel_size, conv_stride, conv_padding, conv_dilation = map(int, conv_params.split("_")[1:])
        except ValueError:
            raise ValueError("Error: conv_params format is incorrect.")

        # Parse max-pooling layer parameters
        try:
            pool_kernel_size, pool_stride, pool_padding, pool_dilation = map(int, maxpool_params.split("_")[1:])
        except ValueError:
            raise ValueError("Error: maxpool_params format is incorrect.")

        # Convolutional layers
        for _ in range(conv_layer_num):
            self.layers.append(nn.Conv1d(in_channels, out_channels, conv_kernel_size, conv_stride, conv_padding, conv_dilation))
            final_size = (final_size + 2 * conv_padding - conv_dilation * (conv_kernel_size - 1) - 1) // conv_stride + 1
            self.layers.append(nn.ReLU())
            self.layers.append(nn.MaxPool1d(pool_kernel_size, pool_stride, pool_padding, pool_dilation))
            final_size = (final_size + 2 * pool_padding - pool_dilation * (pool_kernel_size - 1) - 1) // pool_stride + 1
            in_channels = out_channels  # Update input channels for the next layer

        # Fully connected layers
        self.fc_layers = nn.ModuleList()
        try:
            fc_layer_num, in_features, out_features = map(int, linear_params.split("_")[1:])
        except ValueError:
            print("Error: linear_params format is incorrect.")
            raise
        # First FC has in features: flat_size * out_channels
        self.fc_layers.append(nn.Linear(out_channels * final_size, out_features))
        self.fc_layers.append(nn.ReLU())
        for i in range(1,int(fc_layer_num)):
            self.fc_layers.append(nn.Linear(in_features, out_features))
            self.fc_layers.append(nn.ReLU())

        # Output layer
        self.output_layer = nn.Linear(out_features, 2)  # Binary classification

    def forward(self, x):
        try:
            x = x.unsqueeze(1)  # Add channel dimension

            # Pass through convolutional layers
            for layer in self.layers:
                x = layer(x)

            # Flatten dynamically
            x = x.view(x.size(0), -1)  # Convert to (batch_size, flattened_features)

            # Fully connected layers
            for layer in self.fc_layers:
                x = layer(x)

            x = torch.softmax(self.output_layer(x), dim=1)
            return x
        except RuntimeError:
            print("Invalid form, skipping now...")
            x = torch.tensor([1]).float()
            x[x == 1] = float('nan')
            return x

### Helper Functions

In [ ]:
''' 
Function that initializes the CNN and its core components.
Inputs:
    1) cnn_num: cnn bit number
Returns:
    1) cnn: model used(ResNet18)
    2) criterion: loss function, current default=CrossEntropyLoss
    3) learning_rate: learning rate, current default=def_lr
    4) optimizer: optimizer, current default=Adam
'''
def cnn_init(conv_param, maxpool_param, linear_param, learning_rate, trace_len):
    new_cnn = DynamicCNN(conv_param, maxpool_param, linear_param, trace_len)
    # Optimizer: not specified in paper, using Adam
    optimizer = optim.Adam(new_cnn.parameters(), lr=learning_rate)
    # Loss function: not specified in paper, using Cross Entropy Loss
    criterion = nn.CrossEntropyLoss()
    # Learning rate: default set to def_lr, adjust accordingly
    learning_rate = def_lr
    return new_cnn, criterion, optimizer
 
''' 
Function that initializes parameters used in training.
Inputs:
    None
Returns:
    TRAINING PARAMETERS
    1) num_epochs: number of maximum epochs per training
    2) max_grad_norm: gradient clipping threshold
'''
def param_init():
    # training parameters
    num_epochs = max_epoch
    max_grad_norm = 1.0
    
    return num_epochs, max_grad_norm

''' 
Function that loads variables from saved .pth file. Checks if the file is valid.
This function simply receives the checkpoint file and checks if it is valid.
Inputs:
    1) checkpoint_path: string of .pth file name 
    2) device: selected device to run training
Returns:
    1) pth_variables: dictionary that contains received .pth variables.
    List of variables are saved in 'load_values' array.
'''
def load_pth_file(checkpoint_path, device, load_values):
    pth_variables = {}
    # Load .pth file
    try:
        checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=True)
        for values in load_values:
            pth_variables[values] = checkpoint.get(values, 0)
        return pth_variables
    except FileNotFoundError:
        print(f"No checkpoint found at {checkpoint_path}. Starting from scratch.")
        raise
    except KeyError as e:
        missing_keys = {
            'cnn_state_dict': "CNN state dictionary",
            'optimizer_state_dict': "optimizer state dictionary",
            'epoch': "epoch",
            'reached_acc': "recorded accuracy of current epoch"
        }
        key = e.args[0]
        if key in missing_keys:
            if key in ['cnn_state_dict', 'optimizer_state_dict', 'epoch', 'reached_acc']:
                print(f"Critical error: Missing \"{key}\" that stores \"{missing_keys[key]}\". Checkpoint file is invalid.")
                raise

        print(f"Unknown parameter \"{key}\" in checkpoint. Recheck file and try again.")
        raise

# Used for plotting over certain window of values
def moving_average(values, window=10):
    if len(values) < window:
        return values
    return np.convolve(values, np.ones(window)/window, mode='valid')

In [ ]:
cnns = []
train_dataloaders = train_builder.dataloaders
for a, b in train_dataloaders[0]:
    trace_len = len(a[0])

In [ ]:
timestamp   = datetime.datetime.now().strftime('%Y%m%d_%H%M')

print(f"Installed CUDA version: {torch.version.cuda}\n")

# Create CNN per dataset
for i in range(adc_num-1, -1, -1):
    for j in range(adc_bitwidth-1, -1, -1):
        print(f"Starting training for \"bit_{i*8+j}\"...")

        # initialize CNN
        learning_rate = def_lr
        cnn, criterion, optimizer = cnn_init(conv_param, maxpool_param, linear_param, learning_rate, trace_len)
        # Print LR
        print(f"Learning rate:{learning_rate}")
        # initialize parameters
        num_epochs, max_grad_norm = param_init()

        # Set device to cuda if available
        if torch.cuda.is_available():
            print("\tGPU found, running training on GPU...")
            device = torch.device("cuda")
            cnn = cnn.to(device)
        else:
            print("\tNo GPU found, running training on CPU...")
            print("\tRecheck CUDA version and if your GPU supports it.")
            device = torch.device("cpu")

        # Append CNN to cnns array
        cnns.append(cnn)

        # Create checkpoint to save progress
        checkpoint_path = f"cnn_5px_adc_{i}_bit_{j}.pth"

        # Attempt to load saved .pth file
        # start_epoch = LOCAL variable that specifies starting epoch number
        # prev_acc = LOCAL variable that specifies accuracy achieved by loaded .pth file
        load_values = ['cnn_state_dict', 'optimizer_state_dict', 'epoch', 'reached_acc']
        try:
            pth_vars = load_pth_file(checkpoint_path, device, load_values)
        except FileNotFoundError:
            start_epoch = 0
            prev_acc = 0
        else:
            # Manually create variables per value
            checkpoint = torch.load(checkpoint_path, map_location=device)
            cnn.load_state_dict(checkpoint['cnn_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch']
            print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
            prev_acc = checkpoint['reached_acc']
        
            print(f"Checkpoint loaded. Resuming from epoch {start_epoch}.")
            print(f"\tPrevious reached accuracy: {prev_acc}.")

        # Skip training if target accuracy is reached
        if prev_acc == 1:
            print(f"\tSkipping training: accuracy of 1 already achieved.\n")
            continue
        
        # Plot arrays
        loss_values = []
        accuracy_values = []
        value_window = 50

        plt.ion()  # Turn on interactive mode
        fig, (ax) = plt.subplots(figsize=(8, 5))

        # Start training
        for e in range(start_epoch, num_epochs):
            correct = 0
            total_traces = 0
            cnn.train()
            # for dataloader_index in range(i*8+j)
            for inputs, labels in train_dataloaders[i*8+j]:
                inputs, labels = inputs.to(device), labels.to(device) 
                optimizer.zero_grad()
                output = cnn(inputs)
                # Check for NaN in outputs
                if torch.isnan(output).any():
                    print("NaN detected in cnn outputs.")
                    break

                loss = criterion(output, labels)
                # Check for NaN in loss
                if torch.isnan(loss):
                    print("NaN detected in loss. Stopping training.")
                    break
                # print(f"Loss: {loss.item()}")
                loss.backward()

                # Gradient clipping
                torch.nn.utils.clip_grad_norm_(cnn.parameters(), max_grad_norm)
                optimizer.step()
                _, predicted = torch.max(output, 1)
                correct += (predicted == labels).sum()

                total_traces += len(labels)
    
            accuracy = correct / total_traces
            loss_values.append(loss.item())
            accuracy_values.append(float(accuracy))
            
            loss_ma = moving_average(loss_values, window=value_window)
            accuracy_ma = moving_average(accuracy_values, window=value_window)

            # Change rate of update for printing accuracy accordingly
            if (e + 1) % 100 == 0:
                print(f'TRAINING: cnn[{i*8+j}] (adc {i}, bit{j}), Epoch {e+1}, Loss: {loss.item()}')
                print(f'TRAINING: cnn[{i*8+j}] (adc {i}, bit{j}), Epoch {e+1}, Accuracy: {accuracy}')
                
            if (e + 1) % 500 == 0:
                # Save checkpoint
                torch.save({
                    'cnn_state_dict': cnn.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'epoch': e + 1,
                    'reached_acc': accuracy
                }, checkpoint_path)
                print(f"Checkpoint saved for epoch {e + 1}")
                clear_output(wait=True)
                ax.clear()
                ax.plot(range(start_epoch + 9, start_epoch + 9 + len(loss_ma)), loss_ma, label=f'Loss ({value_window}-epoch avg)', color='black')
                ax.plot(range(start_epoch + 9, start_epoch + 9 + len(accuracy_ma)), accuracy_ma, label=f'Accuracy ({value_window}-epoch avg)', color='red')
                ax.set_xlabel('Epoch')
                ax.set_ylabel('Los/Accuracy')
                ax.set_title(f'Training Loss/Accuracy ({value_window}-epoch moving average)')
                ax.legend()

                display(fig)
                plt.pause(0.1)

            prev_acc = accuracy

            if accuracy >= 0.95:
                print(f"Reached accuracy of 1.0. Stopping training for \"cnn[{i*8+j}] (adc {i}, bit{j})\".\n")
                break

### Testing the CNNs

Using the resulting CNNs saved in the .pth files, we test each CNNs using the provided power traces.

Currently the testing data is a subset of the training data. In the future, if we can generate more data, we will be able to use separate datasets.



#### Simple Test Accuracy

Prints single float value for testing accuracy.

In [ ]:
# Create TESTING dataloaders
test_dataloaders = test_builder.dataloaders
track_running_stats=False
# Run evaluation using testing data
for i in range(adc_num-1, -1, -1):
    for j in range(adc_bitwidth-1, -1, -1):
        cnn = cnns[i*8+j]
        try:
            checkpoint_path = f"cnn_5px_adc_{i}_bit_{j}.pth"
            checkpoint = torch.load(checkpoint_path, map_location=device)
            print(checkpoint.keys())
            cnn.load_state_dict(checkpoint['cnn_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch']
            print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
            reached_target_acc = checkpoint['reached_acc']
            print(f"Previous reached accuracy: {reached_target_acc}")
            '''
            if reached_target_acc != 1:
                print(f"cnn_{i}\" did not reach accuracy of 1, skipping evaluation.\n\n")
                continue
            '''
        except FileNotFoundError:
            print(f"No checkpoint found for \"cnn_{i}\". Skipping evaluation.")
        
        correct = 0
        total = 0
        with torch.no_grad():
            # Use TEST dataloader
            for inputs, labels in test_dataloaders[i*8+j]:
                # inputs = inputs.unsqueeze(1).unsqueeze(-1)
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = cnn(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        print(f"Accuracy: {100 * correct / total:.2f}%\n")

#### Classification report

Creates a classification report on all 8 bits.



In [ ]:
# Necessary imports
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, RocCurveDisplay
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from itertools import cycle

In [ ]:
all_preds = []
all_labels = []
all_logits = []  # To store raw logits

for i in range(adc_num-1, -1, -1):
    for j in range(adc_bitwidth-1, -1, -1):
        cnn = cnns[i*8+j]
        print(f"{i*8+j}")
        # Loss function: not specified in paper, using Cross Entropy Loss
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(cnn.parameters(), lr=5e-3)

        if torch.cuda.is_available():
            device = torch.device("cuda")
            cnn = cnn.to(device)
        else:
            device = torch.device("cpu")

        checkpoint_path = f"resnet18_checkpoint_adc_{i}_bit_{j}.pth"
        try:
            checkpoint = torch.load(checkpoint_path, map_location=device)
            cnn.load_state_dict(checkpoint['cnn_state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            start_epoch = checkpoint['epoch']
            # print(f"Checkpoint loaded. Resuming from epoch {start_epoch}")
            reached_target_acc = checkpoint['reached_acc']
            # print(f"Previous reached accuracy: {reached_target_acc}")
            '''
            if reached_target_acc != 1:
                print(f"cnn_{i}\" did not reach accuracy of 1, skipping evaluation.\n\n")
                continue
            '''
        except FileNotFoundError:
            print(f"No checkpoint found for \"cnn_{i}\". Skipping evaluation.")
            continue

        # Evaluate the model
        with torch.no_grad():
            # Use TEST dataloader
            for inputs, labels in test_dataloaders[i*8+j]:
                # inputs = inputs.unsqueeze(1).unsqueeze(-1)
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = cnn(inputs)  # Raw logits
                _, preds = torch.max(outputs, 1)

                adc_logits = outputs.cpu().numpy()  # Store raw logits
                adc_preds = preds.cpu().numpy()
                adc_labels = labels.cpu().numpy()

        all_logits.append(adc_logits)
        all_preds.append(adc_preds)
        all_labels.append(adc_labels)
    
        # Compute accuracy for adc_i
        accuracy = accuracy_score(adc_labels, adc_preds)
        print(f"Test Accuracy for adc_{i}: {accuracy:.4f}")

        # Generate classification report for adc_i
        print(f"\nClassification Report for adc_{i}:")
        print(classification_report(adc_labels, adc_preds))

        # Generate confusion matrix for adc_i
        print(f"\nConfusion Matrix for adc_{i}:")
        print(confusion_matrix(adc_labels, adc_preds))

all_logits = np.concatenate(all_logits)
all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

print("\nFINAL COMBINED RESULTS FOR ALL ADCS")

# Compute accuracy for all adcs
accuracy = accuracy_score(all_labels, all_preds)
print(f"Test Accuracy: {accuracy:.4f}")

# Generate classification report for all adcs
print("\nClassification Report:")
print(classification_report(all_labels, all_preds))

# Generate confusion matrix for all adcs
print("\nConfusion Matrix:")
print(confusion_matrix(all_labels, all_preds))